---

_You are currently looking at **version 1.2** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-social-network-analysis/resources/yPcBs) course resource._

---

# Assignment 4

In [1]:
import networkx as nx
import pandas as pd
import numpy as np
import pickle

---

## Part 1 - Random Graph Identification

For the first part of this assignment you will analyze randomly generated graphs and determine which algorithm created them.

In [4]:
P1_Graphs = pickle.load(open('A4_graphs','rb'))
P1_Graphs

<br>
`P1_Graphs` is a list containing 5 networkx graphs. Each of these graphs were generated by one of three possible algorithms:
* Preferential Attachment (`'PA'`)
* Small World with low probability of rewiring (`'SW_L'`)
* Small World with high probability of rewiring (`'SW_H'`)

Anaylze each of the 5 graphs and determine which of the three algorithms generated the graph.

*The `graph_identification` function should return a list of length 5 where each element in the list is either `'PA'`, `'SW_L'`, or `'SW_H'`.*

In [9]:
def graph_identification():
    
    algos = []
    
    for g in P1_Graphs:
        
        clustering = nx.average_clustering(g)
        shortest_path = nx.average_shortest_path_length(g)
        
        degrees = g.degree()
        dg_values = sorted(set(degrees.values()))
        dg_hist = [list(degrees.values()).count(i) / float(nx.number_of_nodes(g)) for i in dg_values]
        
        if len(dg_hist) > 10:
            
            algos.append('PA')
        
        elif clustering < 0.1:
            
            algos.append('SW_H')
        
        else:
            
            algos.append('SW_L')
    
    return algos

---

## Part 2 - Company Emails

For the second part of this assignment you will be workking with a company's email network where each node corresponds to a person at the company, and each edge indicates that at least one email has been sent between two people.

The network also contains the node attributes `Department` and `ManagementSalary`.

`Department` indicates the department in the company which the person belongs to, and `ManagementSalary` indicates whether that person is receiving a management position salary.

In [11]:
G = nx.read_gpickle('email_prediction.txt')

print(nx.info(G))

Name: 
Type: Graph
Number of nodes: 1005
Number of edges: 16706
Average degree:  33.2458


### Part 2A - Salary Prediction

Using network `G`, identify the people in the network with missing values for the node attribute `ManagementSalary` and predict whether or not these individuals are receiving a management position salary.

To accomplish this, you will need to create a matrix of node features using networkx, train a sklearn classifier on nodes that have `ManagementSalary` data, and predict a probability of the node receiving a management salary for nodes where `ManagementSalary` is missing.



Your predictions will need to be given as the probability that the corresponding employee is receiving a management position salary.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC).

Your grade will be based on the AUC score computed for your classifier. A model which with an AUC of 0.88 or higher will receive full points, and with an AUC of 0.82 or higher will pass (get 80% of the full points).

Using your trained classifier, return a series of length 252 with the data being the probability of receiving management salary, and the index being the node id.

    Example:
    
        1       1.0
        2       0.0
        5       0.8
        8       1.0
            ...
        996     0.7
        1000    0.5
        1001    0.0
        Length: 252, dtype: float64

In [17]:
def salary_predictions():
    
    from sklearn.preprocessing import MinMaxScaler
    from sklearn.ensemble import RandomForestClassifier
    
    df = pd.DataFrame(index=G.nodes())
    
    df['Department'] = pd.Series(nx.get_node_attributes(G, 'Department'))
    df['ManagementSalary'] = pd.Series(nx.get_node_attributes(G, 'ManagementSalary'))
    
    df['Degree'] = pd.Series(G.degree())
    df['Closeness'] = pd.Series(nx.closeness_centrality(G, normalized=True))
    df['Betweenness'] = pd.Series(nx.betweenness_centrality(G, normalized=True))
    df['Clustering'] = pd.Series(nx.clustering(G))
    df['Degree_centrality'] = pd.Series(nx.degree_centrality(G))
    df['Closeness_centrality'] = pd.Series(nx.closeness_centrality(G))
    df['PageRank'] = pd.Series(nx.pagerank(G))
    
    Train_data = df[~df['ManagementSalary'].isnull()]
    Test_data = df[df['ManagementSalary'].isnull()]
    
    features = ['Degree', 'Closeness', 'Betweenness', 'Clustering', 'Degree_centrality', 'Closeness_centrality', 'PageRank']
    
    X_train = Train_data[features]
    y_train = Train_data['ManagementSalary']
    
    X_test = Test_data[features]
    
    scaler = MinMaxScaler().fit(X_train)
    
    X_train_scl = scaler.transform(X_train)
    X_test_scl = scaler.transform(X_test)
    
    model = RandomForestClassifier(n_estimators=100, max_depth=10).fit(X_train_scl, y_train)
    
    preds = (model.predict_proba(X_test_scl)[:, 1]).tolist()
    
    return pd.Series(preds, X_test.index)

1       0.016567
2       0.960000
5       1.000000
8       0.309051
14      0.096250
18      0.098254
27      0.065687
30      0.757381
31      0.331900
34      0.068710
37      0.070000
40      0.069279
45      0.015016
54      0.331605
55      0.575726
60      0.204854
62      1.000000
65      0.990000
77      0.034651
79      0.042494
97      0.010000
101     0.000000
103     0.445034
108     0.037875
113     0.100000
122     0.000000
141     0.235000
142     1.000000
144     0.000505
145     0.422500
          ...   
913     0.000000
914     0.000505
915     0.000000
918     0.050000
923     0.000000
926     0.050505
931     0.010505
934     0.000000
939     0.000000
944     0.000000
945     0.000000
947     0.010833
950     0.068749
951     0.000000
953     0.000000
959     0.000000
962     0.000000
963     0.124436
968     0.000833
969     0.002886
974     0.000000
984     0.000000
987     0.010000
989     0.000833
991     0.000833
992     0.000000
994     0.000000
996     0.0000

### Part 2B - New Connections Prediction

For the last part of this assignment, you will predict future connections between employees of the network. The future connections information has been loaded into the variable `future_connections`. The index is a tuple indicating a pair of nodes that currently do not have a connection, and the `Future Connection` column indicates if an edge between those two nodes will exist in the future, where a value of 1.0 indicates a future connection.

In [18]:
future_connections = pd.read_csv('Future_Connections.csv', index_col=0, converters={0: eval})
future_connections.head(10)

Future Connection
(6, 840)                  0.0
(4, 197)                  0.0
(620, 979)                0.0
(519, 872)                0.0
(382, 423)                0.0
(97, 226)                 1.0
(349, 905)                0.0
(429, 860)                0.0
(309, 989)                0.0
(468, 880)                0.0

Using network `G` and `future_connections`, identify the edges in `future_connections` with missing values and predict whether or not these edges will have a future connection.

To accomplish this, you will need to create a matrix of features for the edges found in `future_connections` using networkx, train a sklearn classifier on those edges in `future_connections` that have `Future Connection` data, and predict a probability of the edge being a future connection for those edges in `future_connections` where `Future Connection` is missing.



Your predictions will need to be given as the probability of the corresponding edge being a future connection.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC).

Your grade will be based on the AUC score computed for your classifier. A model which with an AUC of 0.88 or higher will receive full points, and with an AUC of 0.82 or higher will pass (get 80% of the full points).

Using your trained classifier, return a series of length 122112 with the data being the probability of the edge being a future connection, and the index being the edge as represented by a tuple of nodes.

    Example:
    
        (107, 348)    0.35
        (542, 751)    0.40
        (20, 426)     0.55
        (50, 989)     0.35
                  ...
        (939, 940)    0.15
        (555, 905)    0.35
        (75, 101)     0.65
        Length: 122112, dtype: float64

In [24]:
def new_connections_predictions():
    
    from sklearn.preprocessing import MinMaxScaler
    from sklearn.ensemble import RandomForestClassifier
    
    future_connections['Preferential_attachment'] = [i[2] for i in nx.preferential_attachment(G, future_connections.index)]
    future_connections['Common_neighbours'] = [len(list(nx.common_neighbors(G, n[0], n[1]))) for n in future_connections.index]
    future_connections['Jaccard_coefficient'] = [i[2] for i in nx.jaccard_coefficient(G, future_connections.index)]
    future_connections['Resources_allocation'] = [i[2] for i in nx.resource_allocation_index(G, future_connections.index)]
    future_connections['Adamic_adar'] = [i[2] for i in nx.adamic_adar_index(G, future_connections.index)]
    
    for n in G.nodes():
        G.node[n]['community'] = G.node[n]['Department']
        
    future_connections['cn_soundarajan_hopcroft'] = [i[2] for i in nx.cn_soundarajan_hopcroft(G, future_connections.index)]
    future_connections['ra_soundarajan_hopcroft'] = [i[2] for i in nx.ra_index_soundarajan_hopcroft(G, future_connections.index)]
    
    Train_data = future_connections[~future_connections['Future Connection'].isnull()]
    Test_data = future_connections[future_connections['Future Connection'].isnull()]
    
    features = ['Preferential_attachment', 'Common_neighbours', 'Jaccard_coefficient', 'Resources_allocation',
                'Adamic_adar', 'cn_soundarajan_hopcroft', 'ra_soundarajan_hopcroft']
    
    X_train = Train_data[features]
    y_train = Train_data['Future Connection']
    
    X_test = Test_data[features]
    
    scaler = MinMaxScaler().fit(X_train)
    
    X_train_scl = scaler.transform(X_train)
    X_test_scl = scaler.transform(X_test)
    
    model = RandomForestClassifier(n_estimators=100, max_depth=10).fit(X_train_scl, y_train)
    
    preds = (model.predict_proba(X_test_scl)[:, 1]).tolist()
    
    return pd.Series(preds, X_test.index)

(107, 348)    0.025795
(542, 751)    0.012540
(20, 426)     0.532198
(50, 989)     0.012587
(942, 986)    0.012717
(324, 857)    0.012618
(13, 710)     0.119911
(19, 271)     0.143930
(319, 878)    0.012491
(659, 707)    0.012625
(49, 843)     0.012591
(208, 893)    0.012596
(377, 469)    0.013750
(405, 999)    0.016026
(129, 740)    0.011889
(292, 618)    0.018544
(239, 689)    0.012491
(359, 373)    0.016044
(53, 523)     0.384360
(276, 984)    0.012204
(202, 997)    0.012547
(604, 619)    0.037699
(270, 911)    0.012491
(261, 481)    0.059934
(200, 450)    0.981548
(213, 634)    0.012527
(644, 735)    0.039979
(346, 553)    0.012475
(521, 738)    0.012218
(422, 953)    0.014367
                ...   
(672, 848)    0.012491
(28, 127)     0.987743
(202, 661)    0.012401
(54, 195)     0.999994
(295, 864)    0.012734
(814, 936)    0.012418
(839, 874)    0.012717
(139, 843)    0.012618
(461, 544)    0.013858
(68, 487)     0.013168
(622, 932)    0.012644
(504, 936)    0.017671
(479, 528) 